Private sampling implementation

In [ ]:
pip install qpsolvers
pip install sdgym==0.4.1

In [12]:
import numpy as np
import math
from copy import deepcopy
from tqdm import tqdm
import scipy.linalg as sln
from scipy.optimize import minimize
from scipy.optimize import linprog
from scipy.special import comb
from time import time
import numpy as np
import random
import qpsolvers
import pandas as pd
from sdgym.datasets import load_dataset, load_tables

In [17]:
def subsets_indices(d,p):
  
      if d==1:
        return [[[i] for i in range(p)]]
      else:
        a = subsets_indices(d-1,p)
        b = deepcopy(a[-1])
        n = len(b)
        for i in range(len(b)):
            for j in range(b[i][-1]+1,p):
                b.append(b[i] + [j])
        b = b[n:]
        return a + [b]

In [65]:
class density:

    def __init__(self, data=None, type='data', space=None, inter=None, distr=None):

        if type == 'data':
          self.data = data
          self.space, self.inter, self.dsy = np.unique(data, axis=0, return_counts=True, return_inverse=True)
          self.dsy = self.dsy*(1/len(data))

        elif type == 'distr':
          self.data = None
          self.space = space
          self.inter = inter
          self.dsy = np.zeros(len(space))
          for i in range(len(distr)):
            self.dsy[inter[i]] += distr[i]
          
        x = self.space.astype(int)
        keys = np.array2string(x, threshold=10**5)
        keys = keys[1:-1]
        keys= keys.split('\n ')
        self.dsyDict = {keys[i] : self.dsy[i] for i in range(len(keys))}


    def normalize(self):

      self.dsy = self.dsy * 1/sum(self.dsy)


    def noise_data(self, epsilon):
      
      noise = np.random.laplace(scale=1/epsilon,size=len(self.space))/(len(self.space))
      dsy = self.dsy
      dsy += noise
      dsy = np.clip(dsy, a_min=0, a_max = np.inf)
      print(dsy)
      self.dsy = dsy * 1/sum(dsy)


    def get_marginal(self,indices,value):
      
      dst = 0
      for i in range(len(self.space)):
        dst += np.array_equal(self.space[i][indices], value)*self.dsy[i]
      return dst


    def get_marginals(self,n):
      indices = subsets_indices(n,len(self.space[0]))
      indices = [item for sublist in indices for item in sublist]
      dsts = np.zeros(len(indices))
      for i in range(len(indices)):
        dsts[i] = self.get_marginal(indices[i],[1]*len(indices[i]))
      return dsts


    def sample(self, n):
      self.normalize()
      rdt = random.choices(np.arange(len(self.space)),self.dsy,k=n)
      return self.space[rdt]


    @staticmethod
    def arePerm(self, x,y):
        if len(x) != len(y):
            return False
        x, y = x.astype(int), y.astype(int)
        keys,tstkeys = np.array2string(x), np.array2string(y)
        keys, tstkeys = keys[1:-1], tstkeys[1:-1]
        keys, tstkeys = keys.split('\n '), tstkeys.split('\n ')
        dct = dict.fromkeys(keys, True)
        for tst in tstkeys:
            if tst not in dct:
                return False
        return True
    

    def dsyToVal(self, input):
        x = input.astype(int)
        key = np.array2string(x)
        if key not in self.dsyDict:
            return 0
        else:
            return self.dsyDict[key]



In [18]:
class PrivateSampler:
    def __init__(self, data, m, d, delta, Delta):

        data = 2*(data-1/2)
        self.data = data
        self.m = m
        self.d = d
        self.n = len(data)
        self.p = len(data[0])
        criteria = math.exp(2*d)*self.sum_comb(self.d,self.p)
        if m>2**self.p:
          print(f"m larger than research space, cut m to {2**self.p}")
          self.m = 2**self.p
        if self.m < criteria:
          print(f'Warning : m = {self.m} should be at least greater than {criteria}')
        print(f'{self.sum_comb(self.d,self.p)} marginals to be fitted with {self.n} rows')
        self.empiricalDsy = density(data=data, type='data')
        max_density = max(self.empiricalDsy.dsy)
        self.Delta = Delta
        if Delta < max_density * (2**self.p) :
          print(f'Be careful : if the density is upper bounded by {max_density}, then Delta > {max_density * (2**self.p)}.')
          print(f"Setting Delta to {max_density * (2**self.p)}")
          self.Delta = max_density * (2**self.p)
        alpha= (2**self.p)*min(self.empiricalDsy.get_marginals(1))**self.p
        if delta < 8*(1/alpha)*math.sqrt(math.exp(2*d)*PrivateSampler.sum_comb(d,self.p)/self.m):
          print(f'Be careful : if the density is bounded below by {min(self.empiricalDsy.get_marginals(1))**self.p}, then Then, delta>{8*(1/alpha)*math.sqrt(math.exp(2*d)*PrivateSampler.sum_comb(d,self.p)/self.m)}.')
        self.empiricalUniformDsy = None
        self.samplerDsy = None
        self.space = None
        self.delta = delta
        

    def fit(self):

      self.create_space()
      Mx = np.transpose(PrivateSampler.walsh_matrix(self.space,self.d))
      Mobj = np.transpose(PrivateSampler.walsh_matrix(self.data,self.d))
      f_obj = np.sum(Mobj, axis=1)
      u_obj = np.sum(Mx, axis=1)
      lbd = self.lambda_fit(Mx, u_obj, f_obj)
      dsy = self.dsy_fit(lbd, Mx, f_obj, u_obj)
      if abs(sum(dsy)-1) > 10**(-5):
        print(f'Warning : density does not sum to 1 ({sum(dsy)})')
      dsy = dsy / sum(dsy)
      self.samplerDsy = density(distr=dsy,type='distr',space=self.empiricalUniformDsy.space,inter=self.empiricalUniformDsy.inter)
      return


    def sample(self,n):

      return self.samplerDsy.sample(n)


    @staticmethod
    def sum_comb(d,p):

      return sum(comb(p,i,exact=True) for i in range(1,d+1))


    @staticmethod
    def walsh(x,J):

      prod = 1
      for i in J:
          prod *= x[i]
      return prod
    

    @staticmethod
    def walsh_matrix(s,d):

      m = len(s)
      p = len(s[0])
      low_freq = sum(comb(p,i,exact=True) for i in range(1,d+1))
      wls = np.zeros((m,low_freq))
      indexes = subsets_indices(d,p)
      indexes = [item for sublist in indexes for item in sublist]
      for i in tqdm(range(m)):
        for j in range(low_freq):
            wls[i,j] = PrivateSampler.walsh(s[i],indexes[j])          
      return wls 


    @staticmethod
    def is_well_conditioned(M,m,d):

      sv = sln.svdvals(M)
      smallest = sv[-1]
      bound = math.sqrt(m)/(2*math.exp(d))
      return smallest > bound


    #https://stackoverflow.com/questions/18296035/how-to-extract-the-bits-of-larger-numeric-numpy-data-types
    def unpackbits(x, num_bits):

      if np.issubdtype(x.dtype, np.floating):
        raise ValueError("numpy data type needs to be int-like")
      xshape = list(x.shape)
      x = x.reshape([-1, 1])
      mask = 2**np.arange(num_bits, dtype=x.dtype).reshape([1, num_bits])
      return (x & mask).astype(bool).astype(int).reshape(xshape + [num_bits])[:,::-1]


    def create_space(self):
      
      conditioned = False
      while not(conditioned):
        draw = np.random.choice(np.arange(2**self.p), size=self.m, replace=False)
        S = PrivateSampler.unpackbits(draw,self.p)
        S = 2*(S-1/2)
        M = PrivateSampler.walsh_matrix(S,self.d)
        conditioned = PrivateSampler.is_well_conditioned(M,self.m,self.d)
      self.space = S
      self.empiricalUniformDsy = density(data=self.space, type='data')
      return


    def lambda_fit(self, Mx, u_obj, f_obj):

      c = np.zeros(self.m+1)
      c[0] = 1
      Wx = np.zeros((len(Mx), len(Mx[0])+1))
      Wx[:,1:] = Mx
      U = np.zeros((len(u_obj),self.m+1))
      F = np.zeros((len(f_obj),self.m+1))
      U[:,0] = u_obj
      F[:,0] = f_obj
      b = u_obj
      A = Wx + F - U

      upperlbd = (self.Delta-self.delta)/self.m
      lowerlbd = 2*self.delta/self.m
      blbd = (lowerlbd,upperlbd)
      bndslbd = ((0,1),) + (blbd,)*self.m

      print('fitting lambda')
      t = time()
      lbd = linprog(c,A_eq=A,b_eq=b,bounds=bndslbd).x[0]
      t = time()- t
      print('in {}s'.format(t))
      print('lambda : {}'.format(lbd))
      return lbd


    def dsy_fit(self, lbd, Mx, f_obj, u_obj):

      match = ((1-lbd)/self.n)*f_obj + (lbd/self.m)*u_obj
      upper = self.Delta/self.m
      lower = self.delta/self.m

      P = np.diag(np.ones(self.m))
      Q = - np.ones(self.m)*1/self.m

      b = np.zeros(len(match)+1)
      b[:-1] = match
      b[-1] = 1
      A = np.zeros((len(Mx)+1, len(Mx[0])))
      A[:-1] = Mx
      A[-1] = np.ones(len(Mx[0]))

      up = np.ones(self.m)*upper
      low = np.ones(self.m)*lower

      t = time()
      print('fitting h')
      sol = qpsolvers.solve_qp(P,Q,A=A,b=b,lb=low,ub=up)
      t = time() - t
      print('in {}s'.format(t))
      return sol

In [13]:
metadata = load_dataset('asia')
tables = load_tables(metadata)
data = tables['asia'].applymap(lambda x : 1 if x=='yes' else 0).to_numpy()

In [27]:
m = 3500
d = 2
Delta=data.shape[0]/2
delta=0.00001

In [15]:
data, data.shape

(array([[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0]], dtype=int64),
 (20000, 8))

In [28]:
sampler = PrivateSampler(data, m, d, delta, Delta)

m larger than research space, cut m to 256
36 marginals to be fitted with 20000 rows
Be careful : if the density is bounded below by 8.16651803662262e-17, then Then, delta>1060311148166572.8.


On this dataset, the sampler finds no solution

In [29]:
sampler.fit()

100%|██████████| 20000/20000 [00:00<00:00, 50642.27it/s]

fitting lambda
in 0.03561568260192871s
lambda : 1.3875566701039697e-07
fitting h
in 0.036997318267822266s


TypeError: 'NoneType' object is not iterable

Trying to fit with delta==0 :

In [66]:
sampler = PrivateSampler(data, m, d, 0, Delta)

m larger than research space, cut m to 256
36 marginals to be fitted with 20000 rows
Be careful : if the density is bounded below by 8.16651803662262e-17, then Then, delta>1060311148166572.8.


In [67]:
sampler.fit()

100%|██████████| 20000/20000 [00:00<00:00, 49128.13it/s]

fitting lambda
in 0.030935287475585938s
lambda : 1.3875566785554005e-07
fitting h
in 0.03824925422668457s


Here, for m big enough, the density computed is close to the empirical density of X on S

In [83]:
empiricalRestricted = np.zeros(sampler.m)
samplerDsy = np.zeros(sampler.m)

In [84]:
for i,j in enumerate(sampler.samplerDsy.dsyDict):
    try:
        empiricalRestricted[i] = sampler.empiricalDsy.dsyDict[j]
    except:
        empiricalRestricted[i] = 0
    samplerDsy[i] = sampler.samplerDsy.dsyDict[j]

In [86]:
empiricalRestricted = empiricalRestricted/sum(empiricalRestricted)

Computing the L2 distance between the empirical density and the sampler density on S :

In [87]:
np.linalg.norm(empiricalRestricted-samplerDsy)

0.03997263958525846